In [ ]:
### Libraries

In [ ]:
import os
from io import BytesIO
import requests
from datetime import datetime,timedelta
import warnings
#import rasterio
import xarray as xr
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import pandas as pd
import matplotlib.patheffects as path_effects
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import ColorbarBase
import matplotlib.colors as mcolors

In [ ]:
def convert_image_to_array(image_stream,crop_box):
    image = Image.open(image_stream)
    image_cropped = image.crop(crop_box)
    if image_cropped.mode == 'P':
        # Convert indexed image to RGB
        image_cropped = image_cropped.convert('RGB')
    image_np = np.array(image_cropped)
    return image_np

def convert_image_to_coordinates(image_to_process, minx, maxx, miny, maxy):
    if isinstance(image_to_process, np.ndarray):
        image = Image.fromarray(image_to_process)
    else:
        image_stream = io.BytesIO(image_to_process)
        image = Image.open(image_stream)
        if image.mode == 'P':
            # Convert indexed image to RGB
            image = image.convert('RGB')
    
    width, height = image.size

    coordinates = []
    values = []
    
    # Loop through each pixel in the image
    for y in range(height):
        for x in range(width):
            # Get the RGB value of the pixel
            pixel_value = image.getpixel((x, y))
            
            # Store the coordinate and pixel value
            coordinates.append((x, y))
            values.append(pixel_value)
    
    # Create a DataFrame with coordinates and RGB values
    df = pd.DataFrame(coordinates, columns=['x', 'y'])
    df[['R', 'G', 'B']] = pd.DataFrame(values, index=df.index)
    
    # Scale x-coordinates from pixel indices to the target range
    df['x_scaled'] = minx + (df['x'] / (width - 1)) * (maxx - minx)
    df['y_scaled'] = miny + (df['y'] / (height - 1)) * (maxy - miny) 

    return df

def convert_df_to_reduced_gdf(df, xsample, ysample):
    gpd_df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.x_scaled, df.y_scaled))
    gpd_df.set_crs(epsg=4326, inplace = True)
    gpd_df = gpd_df[['R','G', 'B', 'geometry']]
    gpd_df = gpd_df.drop_duplicates()
    # Drop every other entry by selecting rows with even indices
    gpd_df_reduced = gpd_df.iloc[::xsample].reset_index(drop=True)
    gpd_df_reduced['x'] = gpd_df_reduced.geometry.x
    gpd_df_reduced['y'] = gpd_df_reduced.geometry.y
    gpd_df_sorted = gpd_df_reduced.sort_values(by = ['x','y'])
    gpd_df_reduced = gpd_df_sorted.iloc[::ysample].reset_index(drop=True)
    return gpd_df_reduced

In [ ]:
def calc_unique_colors(image_array):
    # Reshape the array to a list of pixels
    pixels = image_array.reshape(-1, 3)
    
    # Find unique colors
    unique_colors = np.unique(pixels, axis=0)
    
    # Limit the number of colors to plot
    max_colors = 16
    unique_colors = unique_colors[:max_colors]
    return unique_colors

In [ ]:
# Function to calculate the Euclidean distance
def euclidean_distance(color1, color2):
    return np.sqrt(np.sum((np.array(color1) - np.array(color2)) ** 2))
    
# Function to find the closest category
def closest_color_category(color, color_categories):
    distances = {category: euclidean_distance(color, rgb) for category, rgb in color_categories.items()}
    return min(distances, key=distances.get)

#convert RGB color values
def convert_color(df):
    df['color'] = df.apply(lambda row: (
        row['R'] / 255.0,  # Normalize Red
        row['G'] / 255.0,  # Normalize Green
        row['B'] / 255.0   # Normalize Blue
    ), axis=1)
    return df

In [ ]:
def transform_geom(geometry):
    
    # Check if the geometry is a MultiPolygon
    if geometry.geom_type == 'MultiPolygon':
        # Create a list to hold the modified polygons
        modified_polygons = []
        
        # Iterate through each polygon in the MultiPolygon
        for polygon in geometry.geoms:
            # Get the exterior coordinates and convert them to a list
            exterior_coords = list(polygon.exterior.coords)
            
            # If you need to modify the coordinates, you can do it here
            for i in range(len(exterior_coords)):
                # Example modification: increase x and y by 1
                exterior_coords[i] = ((exterior_coords[i][0] + 360)%360, exterior_coords[i][1])
            
            # Create a new Polygon with modified coordinates
            modified_polygon = Polygon(exterior_coords, [list(interior.coords) for interior in polygon.interiors])
            modified_polygons.append(modified_polygon)
        
        # Create a new MultiPolygon with modified polygons
        modified_multipolygon = MultiPolygon(modified_polygons)
        
        return modified_multipolygon
    else:
        print("The geometry is not a MultiPolygon.")


In [ ]:
shapefile_dir = 'C:\\Users\\Katie.Kowal\\Desktop\\pacific_python\\shapefiles'
spei_url = "https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPEI_1mo.png"
fiji = gpd.read_file(os.path.join(shapefile_dir, 'geoBoundaries-FJI-ADM0-all', 'geoBoundaries-FJI-ADM0.shp'))
solomons = gpd.read_file(os.path.join(shapefile_dir, 'slb_admbnda_adm0', 'slb_admbnda_adm0.shp'))
kiribati = gpd.read_file(os.path.join(shapefile_dir, 'kir_adm_2020_shp', 'kir_admbnda_adm1_2020.shp'))
fiji_geom = fiji['geometry'].simplify(tolerance=0.1).union_all()
solomons_geom = solomons['geometry'].to_crs(fiji.crs).simplify(tolerance=0.1).union_all()
kiribati_groups = ['Gilbert Islands', 'Line Islands', 'Phoenix Islands']
for k in kiribati_groups:
    simple_geom = kiribati.loc[kiribati['ADM1_EN'] == k, 'geometry'].simplify(tolerance = 0.005).union_all()
    kiribati.loc[kiribati['ADM1_EN'] == k, 'geometry'] = simple_geom
kiribati = kiribati[['ADM1_EN', 'geometry']]
fiji = fiji[['shapeName', 'geometry']]
fiji['geometry'] = fiji_geom
fiji['geometry'].iloc[0] = transform_geom(fiji['geometry'].iloc[0])
fiji.columns = ['ADM1_EN', 'geometry']
solomons = solomons[['COUNTRY', 'geometry']].to_crs(kiribati.crs)
solomons['geometry'] = solomons_geom
solomons.columns = ['ADM1_EN', 'geometry']
pac_countries = pd.concat([fiji, solomons], 
                          axis = 0, ignore_index = True)
pac_countries

In [ ]:
fiji_rec = Polygon([(177, -20), (177, -16), (182, -16), (182, -20), (177,-20)])
df_fiji = df_spei_pac[df_spei_pac.geometry.within(fiji.geometry.iloc[0], align = True)]#fiji_rec, align=True)]
no_black = (df_fiji['R'] != 0) & (df_fiji['G'] != 0) & (df_fiji['B'] != 0)
df_fiji = df_fiji[no_black]

In [ ]:
fiji_rec = Polygon([(177, -20), (177, -16), (182, -16), (182, -20), (177,-20)])
df_gilbert = df_spei_pac[df_spei_pac.geometry.within(kiribati.geometry.iloc[2], align = True)]#fiji_rec, align=True)]
no_black = (df_gilbert['R'] != 0) & (df_gilbert['G'] != 0) & (df_gilbert['B'] != 0)
df_gilbert = df_gilbert[no_black]

In [ ]:
def flatten_to_2d(multipolygon_z):
    """
    Flatten a 3D MultiPolygon (Z) to a 2D MultiPolygon.
    This removes the Z-coordinate and returns a new MultiPolygon.
    """
    if not isinstance(multipolygon_z, MultiPolygon):
        raise ValueError("The input must be a MultiPolygon object.")
    
    # List to store the new flattened polygons
    new_polygons = []
    
    # Iterate over each polygon in the MultiPolygon
    for polygon in multipolygon_z.geoms:
        # Get the exterior (outer boundary) without the Z dimension
        exterior_2d = [(x, y) for x, y, z in polygon.exterior.coords]
        
        # Get the interiors (holes) without the Z dimension
        interiors_2d = [
            [(x, y) for x, y, z in interior.coords] for interior in polygon.interiors
        ]
        
        # Create a new 2D polygon and append it to the list
        new_polygons.append(Polygon(exterior_2d, interiors_2d))
    
    # Return a new MultiPolygon with 2D polygons
    return MultiPolygon(new_polygons)

# Example usage:
# Assuming you have a 3D MultiPolygon `multipolygon_z`
# multipolygon_z = ... (your MultiPolygon Z object)

# Flatten it to 2D
solomons_flat = flatten_to_2d(solomons.geometry.iloc[0])
solomons_flat = solomons_flat.simplify(tolerance = 0.7)

In [ ]:
df_solomons = df_spei_pac[df_spei_pac.geometry.within(solomons_flat, align = True)]

In [ ]:
solomons_rec = Polygon([(155, -13), (155, -5), (165, -5), (165, -13), (155,-13)])
df_solomons = df_spei_pac[df_spei_pac.geometry.within(solomons_rec, align = True)]
no_black = (df_solomons['R'] != 0) & (df_solomons['G'] != 0) & (df_solomons['B'] != 0)
df_solomons = df_solomons[no_black]

In [ ]:
df_solomons.plot(color = df_solomons['color'])

In [ ]:
spei_path = BytesIO(requests.get(spei_url).content)
spei_colorbar = convert_image_to_array(spei_path, (150, 650, 900, 660))
spei_colors = calc_unique_colors(spei_colorbar)
spei_colors = {closest_color_category(color, color_categories): tuple(color.tolist()) for color in spei_colors}
pac_cropw = (45,65, 676, 555) #65
pac_crope = (676,65, 1004, 555)
pac_coordsw = (132, 180, 9, -22) #is 10 
pac_coordse = (180, 205, 9, -22)
spei_pacw = convert_image_to_array(spei_path, pac_cropw)
spei_pace = convert_image_to_array(spei_path, pac_crope)
spei_pacw = convert_image_to_coordinates(spei_pacw, pac_coordsw[0], pac_coordsw[1], pac_coordsw[2], pac_coordsw[3])
spei_pace = convert_image_to_coordinates(spei_pace, pac_coordse[0], pac_coordse[1], pac_coordse[2], pac_coordse[3])
spei_pac = pd.concat([spei_pacw, spei_pace], ignore_index = True)
df_spei_pac = convert_df_to_reduced_gdf(spei_pac, 1, 1)
df_spei_pac = convert_color(df_spei_pac)
df_spei_pac.x = (df_spei_pac.x + 360)%360

In [ ]:
convert_spei_color = {
    'dark_red': -2,
    'bright_red': -1.5,
    'orange': -1.2,
    'yellow_orange': -0.8,
    'yellow': -0.4,
    'white': 0,
    'light_blue': 0.4,
    'lightmed_blue': 0.8,
    'med_blue': 1.2,
    'dark_blue': 1.5,
    'navy': 2
}

color_categories = {
    'black' : [0,0,0],
    'dark_red': [170,0,0],
    'bright_red': [255, 30, 0],
    'salmon':[230,140,140],
    'light_pink':[255,230,230],
    'orange': [255, 160, 0],
    'yellow_orange': [255, 192,60],
    'yellow': [255,240,140],
    'dark_brown': [120,80,70],
    'med_brown':[180,140,130],
    'light_brown':[240,220,210],
    'white':[255,255,255],
    'light_green':[200,255,190],
    'bright_green':[120,245,115],
    'dark_green':[30,180,30],
    'light_blue':[180,240,250],
    'lightmed_blue':[150,200,250],
    'med_blue':[80,160,245],
    'dark_blue':[35,120,235],
    'navy': [45,30,165],
    'light_purple':[220,220,255],
    'med_purple':[160,140,255],
    'dark_purple':[120,105,230],
}

In [ ]:
import matplotlib.patches as mpatches
image = Image.open(spei_path)
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(image)
ax.axis('off')
fiji_inset = [0.74, 0.3, 0.15, 0.2]  # [x0, y0, width, height] in figure coordinates
fiji_ax = fig.add_axes(fiji_inset)
fiji_ax.set_xticks([])
fiji_ax.set_yticks([])
#fiji_ax = inset_axes(ax, width="20%", height="20%", loc=4)  # loc=2 means upper left corner
df_fiji.plot(ax = fiji_ax,color = df_fiji['color'], markersize =25)
fiji.geometry.plot(ax = fiji_ax, color = 'none', edgecolor = 'black', linewidth = 2) 
fiji_ax.set_ylim([-18.5,-16])
fiji_ax.set_xlim([177,180.5])

fiji_ax.annotate(
    'Fiji',  # Text
    xy=(0.05, 0.95),             # Position of the annotation (fractional coordinates)
    xycoords='axes fraction',    # Use axes fraction coordinates (relative position)
    fontsize=10,                 # Font size of the annotation
    fontstyle='italic',          # Italic font style
    fontweight='bold',           # Bold font weight
    ha='left',                   # Horizontal alignment of text
    va='top'                     # Vertical alignment of text
)

solomons_inset = [0.73, 0.43, 0.16, 0.2]  # [x0, y0, width, height] in figure coordinates
solomons_ax = fig.add_axes(solomons_inset)
solomons_ax.set_xticks([])
solomons_ax.set_yticks([])
df_solomons.plot(ax = solomons_ax,color = df_solomons['color'], markersize =25)
solomons.geometry.plot(ax = solomons_ax, color = 'none', edgecolor = 'black', linewidth = 2) 
solomons_ax.set_ylim([-12.2,-5.8])
solomons_ax.set_xlim([155,163])

solomons_ax.annotate(
    'Solomons',  # Text
    xy=(0.05, 0.95),             # Position of the annotation (fractional coordinates)
    xycoords='axes fraction',    # Use axes fraction coordinates (relative position)
    fontsize=10,                 # Font size of the annotation
    fontstyle='italic',          # Italic font style
    fontweight='bold',           # Bold font weight
    ha='left',                   # Horizontal alignment of text
    va='top'                     # Vertical alignment of text
)

plt.show()
